In [ ]:
# preparation 
# 2017 ACS 5-year HHs;    https://www2.census.gov/programs-surveys/acs/data/pums/2017/5-Year/csv_hmi.zip
# 2017 ACS 5-year Persons;    https://www2.census.gov/programs-surveys/acs/data/pums/2017/5-Year/csv_pmi.zip
# change downloaded files to "acs2017_psam_h26.csv" and "acs2017_psam_p26.csv"

In [ ]:
import pandas as pd

In [ ]:
puma_county =  pd.read_csv("data/2010_Census_Tract_to_2010_PUMA.txt")
puma_county = puma_county.loc[puma_county.STATEFP == 26]

oakland_puma = list(puma_county.loc[puma_county.COUNTYFP == 125].PUMA5CE.unique())
print(oakland_puma)

In [ ]:
h_pums =  pd.read_csv("data/ACS/acs2017_psam_h26.csv", index_col="SERIALNO")
p_pums = pd.read_csv("data/ACS/acs2017_psam_p26.csv", index_col="SERIALNO")
#print (h_pums.head())
#print (p_pums.head())

In [ ]:
oak_h_pums = h_pums.loc[h_pums.PUMA.isin(oakland_puma)]
oak_p_pums = p_pums.loc[p_pums.PUMA.isin(oakland_puma)]
oak_h_pums = oak_h_pums.loc[(oak_h_pums.TYPE == 1) & (oak_h_pums.NP > 0 )] #remove group quarters and empty units
oak_p_pums = oak_p_pums.loc[oak_h_pums.index]

print (len(oak_h_pums), len(oak_p_pums))

In [ ]:
# add AGEHOH to PUMS sample
hage = oak_p_pums.loc[oak_p_pums.RELP == 0].groupby('SERIALNO').AGEP.max()
oak_h_pums.loc[hage.index, 'AGEHOH'] = hage.values

In [ ]:
# add HRACE to PUMS sample
hrace = oak_p_pums.loc[oak_p_pums.RELP == 0].groupby('SERIALNO').RAC1P.max()
oak_h_pums["v0"] = hrace.values
oak_h_pums["HRACE"] = 4
oak_h_pums.loc[oak_h_pums.v0 == 1, "HRACE"] = 1
oak_h_pums.loc[oak_h_pums.v0 == 2, "HRACE"] = 2
oak_h_pums.loc[oak_h_pums.v0 == 6, "HRACE"] = 3


In [ ]:
# add HHISP to PUMS sample
hisp = oak_p_pums.loc[oak_p_pums.RELP == 0].groupby('SERIALNO').HISP.max()
oak_h_pums["v0"] = hisp.values
oak_h_pums["HHISP"] = 0
oak_h_pums.loc[oak_h_pums.v0 > 1, "HHISP"] = 1


In [ ]:
# add HWORKERS to PUMS sample
workers = oak_p_pums.loc[oak_p_pums.ESR.isin([1,2,4,5])].groupby('SERIALNO').size()
oak_h_pums['v0'] = 0
oak_h_pums.loc[workers.index, "v0"] = workers.values
oak_h_pums["HWORKERS"] = 0
oak_h_pums.loc[oak_h_pums.v0 == 1, "HWORKERS"] = 1
oak_h_pums.loc[oak_h_pums.v0 >= 2, "HWORKERS"] = 2



In [ ]:
for v in ["AGEHOH","HRACE", "HHISP","HWORKERS" ]:
    print (sorted(oak_h_pums[v].unique()))
    print (oak_h_pums.groupby(v).size())
oak_h_pums.drop("v0", axis = 1, inplace = True)

In [ ]:
#HRACE: p_pums[p_pums.RELP == 0].groupby('serialno').RAC1P.max()  1:1, 2:2 3:6 4: .isin([3,4,5,7,8,9]
#HISP: h_pums['hispanic_head'] = p_pums[p_pums.RELP == 0].groupby('serialno').HISP.max() 1:1, 2: >1,,,,,
#WORKERS: p_pums[p_pums.ESR.isin([1,2,4, 5])].groupby('serialno').size(),,,

In [ ]:
oak_h_pums['hh_id'] = oak_h_pums.index.values
oak_p_pums['hh_id'] = oak_p_pums.index.values

In [ ]:
oak_h_pums.to_csv("data/oak_seed_households.csv")
oak_p_pums.to_csv("data/oak_seed_persons.csv")

In [ ]:
oak_h_pums.head()